# Restaurant Revenue Prediction

In fair Kaggle, where we lay our scene,  
Two datasets merge, their features yet unseen.  
Through Random Forest's mighty algorithm's grace,  
Restaurant revenues find their rightful place.  
With dates transformed and categories made clear,  
The model learns what profits may appear.  
Thus trained on truth, it doth predict with care,  
What fortunes future venues might declare.

Dataset: https://www.kaggle.com/competitions/restaurant-revenue-prediction/data

Hugging Face: https://huggingface.co/spaces/alperugurcan/Restaurant-Revenue-Prediction


In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

test = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/test.csv.zip')
train = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/train.csv.zip')

test_ids = test['Id']

test['revenue'] = np.nan

train = train[train['revenue'] < 16000000]

combined = pd.concat([train, test], axis=0, ignore_index=True)

date_column = [col for col in combined.columns if 'date' in col.lower() or 'open' in col.lower()][0]
combined['date'] = pd.to_datetime(combined[date_column])
combined['day'] = combined['date'].dt.day
combined['month'] = combined['date'].dt.month.astype('category')
combined['year'] = combined['date'].dt.year.astype('category')

end_date = pd.to_datetime('2020-08-13')
combined['days'] = (end_date - combined['date']).dt.days

columns_to_drop = ['Id', date_column, 'date']
combined = combined.drop(columns=columns_to_drop)

categorical_columns = combined.select_dtypes(include=['object', 'category']).columns
combined = pd.get_dummies(combined, columns=categorical_columns)

train_final = combined[~combined['revenue'].isna()]
test_final = combined[combined['revenue'].isna()].drop(columns=['revenue'])

np.random.seed(17)
X = train_final.drop('revenue', axis=1)
y = train_final['revenue']

rf_model = RandomForestRegressor(random_state=17)
rf_model.fit(X, y)

predictions = rf_model.predict(test_final)
submission = pd.DataFrame({
    'Id': test_ids,
    'Prediction': predictions
})

submission.to_csv('submission.csv', index=False)

In [10]:
import joblib
joblib.dump(rf_model, 'restaurant_revenue_model.joblib')

['restaurant_revenue_model.joblib']